# Optimization Attempt 1

## Preprocessing

In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf

#  Import and read the charity_data.csv.
import pandas as pd 
application_df1 = pd.read_csv("./charity_data.csv")
application_df1.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [2]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df1 = application_df1.drop(columns={'EIN', 'NAME'})

In [3]:
# Determine the number of unique values in each column.
print(application_df1.nunique())

APPLICATION_TYPE            17
AFFILIATION                  6
CLASSIFICATION              71
USE_CASE                     5
ORGANIZATION                 4
STATUS                       2
INCOME_AMT                   9
SPECIAL_CONSIDERATIONS       2
ASK_AMT                   8747
IS_SUCCESSFUL                2
dtype: int64


In [4]:
# Look at APPLICATION_TYPE value counts for binning
application_df1['APPLICATION_TYPE'].value_counts()

T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: APPLICATION_TYPE, dtype: int64

In [5]:
app_numbers1 = application_df1['APPLICATION_TYPE'].value_counts()
app_numbers1

T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: APPLICATION_TYPE, dtype: int64

### Changing bin size to differentiate more data

In [7]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`
application_types_to_replace1= list(app_numbers1[app_numbers1 < 500].index)

In [8]:
# Replace in dataframe
for app1 in application_types_to_replace1:
    application_df1['APPLICATION_TYPE'] = application_df1['APPLICATION_TYPE'].replace(app1,"Other")

# Check to make sure binning was successful
application_df1['APPLICATION_TYPE'].value_counts()

T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: APPLICATION_TYPE, dtype: int64

In [9]:
# Look at CLASSIFICATION value counts for binning
classification_numbers1 = application_df1["CLASSIFICATION"].value_counts()
classification_numbers1

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C4500        1
C8210        1
C2561        1
C2380        1
C1245        1
Name: CLASSIFICATION, Length: 71, dtype: int64

In [10]:
# You may find it helpful to look at CLASSIFICATION value counts >1
classification_numbers1 = classification_numbers1[classification_numbers1 >= 1]
classification_numbers1

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C4500        1
C8210        1
C2561        1
C2380        1
C1245        1
Name: CLASSIFICATION, Length: 71, dtype: int64

### Changing bin size to differentiate more data

In [14]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`
classifications_to_replace1 = list(classification_numbers1[classification_numbers1 < 1000].index)

In [15]:
# Replace in dataframe
for cls1 in classifications_to_replace1:
    application_df1["CLASSIFICATION"] = application_df1["CLASSIFICATION"].replace(cls1, "Other")
    
# Check to make sure bucketing was successful
application_df1["CLASSIFICATION"].value_counts()

C1000    17326
C2000     6074
C1200     4837
Other     2261
C3000     1918
C2100     1883
Name: CLASSIFICATION, dtype: int64

In [18]:
# Convert categorical data to numeric with `pd.get_dummies`
dummy_df1=pd.get_dummies(application_df1)


In [19]:
# Split our preprocessed data into our features and target arrays
X1=dummy_df1.drop(columns={'IS_SUCCESSFUL'}, axis=1).values
y1=dummy_df1['IS_SUCCESSFUL'].values.reshape(-1, 1)
# Split the preprocessed data into a training and testing dataset
X_train1, X_test1, y_train1, y_test1 = train_test_split(X1, y1, random_state=14)

In [20]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler1 = scaler.fit(X_train1)

# Scale the data
X_train_scaled1 = X_scaler1.transform(X_train1)
X_test_scaled1 = X_scaler1.transform(X_test1)

## Compile, Train and Evaluate the Model

In [21]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features1 = len(X_train_scaled1[0])
hidden_nodes_layer1_1 = 50
hidden_nodes_layer2_1 = 25
# hidden_nodes_layer3 = 25

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer1_1, input_dim=number_input_features1, activation="relu"))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2_1, activation="relu"))

# Third hidden layer
# nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer3, activation="relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))  # binary clasifier -> 1 output neuron/sigmoid activation

# Check the structure of the model
nn.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 50)                2200      
                                                                 
 dense_1 (Dense)             (None, 25)                1275      
                                                                 
 dense_2 (Dense)             (None, 1)                 26        
                                                                 
Total params: 3,501
Trainable params: 3,501
Non-trainable params: 0
_________________________________________________________________


In [22]:
# Compile the model
optimizer1 = tf.keras.optimizers.Adam(learning_rate=0.001)
nn.compile(loss="binary_crossentropy", optimizer=optimizer1, metrics=["accuracy"])

In [23]:
# Add model weight checkpoints during training
import os
from tensorflow.keras.callbacks import ModelCheckpoint
os.makedirs("checkpoints/", exist_ok=True)
checkpoint_path="checkpoints/weights.{epoch}.hdf5"

# Create callback that saves the model's weights every 5 epochs
cp_callback = ModelCheckpoint(filepath=checkpoint_path,
                              verbose=1,
                              save_weights_only=True,
                              period=5,
                              )

In [25]:
# Train the model
fit_model = nn.fit(X_train_scaled1,y_train1,epochs=20,callbacks=[cp_callback])

Epoch 1/20
804/804 [==============================] - 3s 2ms/step - loss: 0.5711 - accuracy: 0.7212
Epoch 2/20
804/804 [==============================] - 2s 2ms/step - loss: 0.5538 - accuracy: 0.7311
Epoch 3/20
804/804 [==============================] - 2s 2ms/step - loss: 0.5508 - accuracy: 0.7322
Epoch 4/20
804/804 [==============================] - 2s 2ms/step - loss: 0.5496 - accuracy: 0.7327
Epoch 5/20
795/804 [============================>.] - ETA: 0s - loss: 0.5480 - accuracy: 0.7342
Epoch 00005: saving model to checkpoints\weights.5.hdf5
804/804 [==============================] - 2s 2ms/step - loss: 0.5478 - accuracy: 0.7340
Epoch 6/20
804/804 [==============================] - 2s 2ms/step - loss: 0.5474 - accuracy: 0.7330
Epoch 7/20
804/804 [==============================] - 2s 2ms/step - loss: 0.5469 - accuracy: 0.7341
Epoch 8/20
804/804 [==============================] - 2s 2ms/step - loss: 0.5465 - accuracy: 0.7325
Epoch 9/20
804/804 [==============================] - 2s 2m

In [26]:
# Evaluate the model using the test data
model_loss1, model_accuracy1 = nn.evaluate(X_test_scaled1,y_test1,verbose=2)
print(f"Loss: {model_loss1}, Accuracy: {model_accuracy1}")

268/268 - 1s - loss: 0.5571 - accuracy: 0.7254 - 642ms/epoch - 2ms/step
Loss: 0.5571255087852478, Accuracy: 0.7253644466400146


# Results 1
#### Model did not achieve 75% or higher.  Making attempt 2.

# Optimization Attempt 2

In [27]:
# Convert categorical data to numeric with `pd.get_dummies`
dummy_df2=pd.get_dummies(application_df1)

In [28]:
# Split our preprocessed data into our features and target arrays
X2=dummy_df2.drop(columns={'IS_SUCCESSFUL'}, axis=1).values
y2=dummy_df2['IS_SUCCESSFUL'].values.reshape(-1, 1)
# Split the preprocessed data into a training and testing dataset
X_train2, X_test2, y_train2, y_test2 = train_test_split(X2, y2, random_state=14)

In [29]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler2 = scaler.fit(X_train2)

# Scale the data
X_train_scaled2 = X_scaler2.transform(X_train2)
X_test_scaled2 = X_scaler2.transform(X_test2)

### Adding another layer in NN

In [30]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features2 = len(X_train_scaled2[0])
hidden_nodes_layer1_2 = 50
hidden_nodes_layer2_2 = 25
hidden_nodes_layer3_2 = 25

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer1_2, input_dim=number_input_features2, activation="relu"))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2_2, activation="relu"))

# Third hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer3_2, activation="relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))  # binary clasifier -> 1 output neuron/sigmoid activation

# Check the structure of the model
nn.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_3 (Dense)             (None, 50)                2200      
                                                                 
 dense_4 (Dense)             (None, 25)                1275      
                                                                 
 dense_5 (Dense)             (None, 25)                650       
                                                                 
 dense_6 (Dense)             (None, 1)                 26        
                                                                 
Total params: 4,151
Trainable params: 4,151
Non-trainable params: 0
_________________________________________________________________


### Making Adam the optimizer

In [31]:
# Compile the model
# optimizer2 = tf.keras.optimizers.Adam(learning_rate=0.001)
nn.compile(loss="binary_crossentropy", optimizer="Adam", metrics=["accuracy"])

In [32]:
# Add model weight checkpoints during training
import os
from tensorflow.keras.callbacks import ModelCheckpoint
os.makedirs("checkpoints/", exist_ok=True)
checkpoint_path="checkpoints/weights.{epoch}.hdf5"

# Create callback that saves the model's weights every 5 epochs
cp_callback = ModelCheckpoint(filepath=checkpoint_path,
                              verbose=1,
                              save_weights_only=True,
                              period=5,
                              )

In [33]:
# Train the model
fit_model2 = nn.fit(X_train_scaled2,y_train2,epochs=20,callbacks=[cp_callback])

Epoch 1/20
804/804 [==============================] - 3s 3ms/step - loss: 0.5732 - accuracy: 0.7200
Epoch 2/20
804/804 [==============================] - 2s 2ms/step - loss: 0.5534 - accuracy: 0.7301
Epoch 3/20
804/804 [==============================] - 2s 3ms/step - loss: 0.5509 - accuracy: 0.7314
Epoch 4/20
804/804 [==============================] - 2s 3ms/step - loss: 0.5487 - accuracy: 0.7327
Epoch 5/20
793/804 [============================>.] - ETA: 0s - loss: 0.5481 - accuracy: 0.7321
Epoch 00005: saving model to checkpoints\weights.5.hdf5
804/804 [==============================] - 2s 3ms/step - loss: 0.5481 - accuracy: 0.7322
Epoch 6/20
804/804 [==============================] - 2s 3ms/step - loss: 0.5466 - accuracy: 0.7327
Epoch 7/20
804/804 [==============================] - 2s 2ms/step - loss: 0.5456 - accuracy: 0.7349
Epoch 8/20
804/804 [==============================] - 2s 3ms/step - loss: 0.5460 - accuracy: 0.7352
Epoch 9/20
804/804 [==============================] - 2s 2m

In [34]:
# Evaluate the model using the test data
model_loss2, model_accuracy2 = nn.evaluate(X_test_scaled2,y_test2,verbose=2)
print(f"Loss: {model_loss2}, Accuracy: {model_accuracy2}")

268/268 - 1s - loss: 0.5550 - accuracy: 0.7242 - 638ms/epoch - 2ms/step
Loss: 0.5550170540809631, Accuracy: 0.7241982221603394


# Results 2
#### Model did not achieve 75% or higher.  Making attempt 3.

# Optimization Attempt 3

In [66]:
# Convert categorical data to numeric with `pd.get_dummies`
dummy_df3=pd.get_dummies(application_df1)

#### Dropped special consideration columns

In [67]:
dummy_df3=dummy_df3.drop(columns={'SPECIAL_CONSIDERATIONS_N','SPECIAL_CONSIDERATIONS_Y'})

In [68]:
# Split our preprocessed data into our features and target arrays
X3=dummy_df3.drop(columns={'IS_SUCCESSFUL'}, axis=1).values
y3=dummy_df3['IS_SUCCESSFUL'].values.reshape(-1, 1)
# Split the preprocessed data into a training and testing dataset
X_train3, X_test3, y_train3, y_test3 = train_test_split(X3, y3, random_state=14)

In [69]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler3 = scaler.fit(X_train3)

# Scale the data
X_train_scaled3 = X_scaler3.transform(X_train3)
X_test_scaled3 = X_scaler3.transform(X_test3)

#### Changing number of features in hidden layers

In [70]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features3 = len(X_train_scaled3[0])
hidden_nodes_layer1_3 = 100
hidden_nodes_layer2_3 = 50
hidden_nodes_layer3_3 = 40

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer1_3, input_dim=number_input_features3, activation="relu"))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2_3, activation="relu"))

# Third hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer3_3, activation="relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))  # binary clasifier -> 1 output neuron/sigmoid activation

# Check the structure of the model
nn.summary()

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_19 (Dense)            (None, 100)               4200      
                                                                 
 dense_20 (Dense)            (None, 50)                5050      
                                                                 
 dense_21 (Dense)            (None, 40)                2040      
                                                                 
 dense_22 (Dense)            (None, 1)                 41        
                                                                 
Total params: 11,331
Trainable params: 11,331
Non-trainable params: 0
_________________________________________________________________


In [71]:
# Compile the model
# optimizer3 = tf.keras.optimizers.Adam(learning_rate=0.001)
nn.compile(loss="binary_crossentropy", optimizer="Adam", metrics=["accuracy"])

In [72]:
# Add model weight checkpoints during training
import os
from tensorflow.keras.callbacks import ModelCheckpoint
os.makedirs("checkpoints/", exist_ok=True)
checkpoint_path3="checkpoints/weights.{epoch}.hdf5"

# Create callback that saves the model's weights every 5 epochs
cp_callback3 = ModelCheckpoint(filepath=checkpoint_path3,
                              verbose=1,
                              save_weights_only=True,
                              period=5,
                              )

#### Adding more epochs

In [73]:
# Train the model
fit_model3 = nn.fit(X_train_scaled3,y_train3,epochs=100,callbacks=[cp_callback3])

Epoch 1/100
804/804 [==============================] - 3s 3ms/step - loss: 0.5668 - accuracy: 0.7240
Epoch 2/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5528 - accuracy: 0.7321
Epoch 3/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5511 - accuracy: 0.7321
Epoch 4/100
804/804 [==============================] - 2s 3ms/step - loss: 0.5490 - accuracy: 0.7336
Epoch 5/100
786/804 [============================>.] - ETA: 0s - loss: 0.5483 - accuracy: 0.7332
Epoch 00005: saving model to checkpoints\weights.5.hdf5
804/804 [==============================] - 2s 3ms/step - loss: 0.5479 - accuracy: 0.7334
Epoch 6/100
804/804 [==============================] - 2s 3ms/step - loss: 0.5462 - accuracy: 0.7334
Epoch 7/100
804/804 [==============================] - 2s 3ms/step - loss: 0.5455 - accuracy: 0.7344
Epoch 8/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5457 - accuracy: 0.7349
Epoch 9/100
804/804 [==============================

804/804 [==============================] - 2s 3ms/step - loss: 0.5344 - accuracy: 0.7416
Epoch 65/100
800/804 [============================>.] - ETA: 0s - loss: 0.5334 - accuracy: 0.7418
Epoch 00065: saving model to checkpoints\weights.65.hdf5
804/804 [==============================] - 2s 3ms/step - loss: 0.5336 - accuracy: 0.7417
Epoch 66/100
804/804 [==============================] - 2s 3ms/step - loss: 0.5343 - accuracy: 0.7405
Epoch 67/100
804/804 [==============================] - 2s 3ms/step - loss: 0.5339 - accuracy: 0.7412
Epoch 68/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5344 - accuracy: 0.7401
Epoch 69/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5335 - accuracy: 0.7412
Epoch 70/100
790/804 [============================>.] - ETA: 0s - loss: 0.5339 - accuracy: 0.7419
Epoch 00070: saving model to checkpoints\weights.70.hdf5
804/804 [==============================] - 2s 2ms/step - loss: 0.5341 - accuracy: 0.7418
Epoch 71/100
80

In [74]:
# Evaluate the model using the test data
model_loss3, model_accuracy3 = nn.evaluate(X_test_scaled3,y_test3,verbose=2)
print(f"Loss: {model_loss3}, Accuracy: {model_accuracy3}")

268/268 - 1s - loss: 0.5764 - accuracy: 0.7242 - 514ms/epoch - 2ms/step
Loss: 0.5764151811599731, Accuracy: 0.7241982221603394


# Results 3
#### Model did not achieve 75% or higher.  It appears saturation is met because model never achieved higher than 72% in test.

In [75]:
# Export final model to HDF5 file
nn.save("Alphabet_Soup_Charity_Optimization.h5")